In [1]:
import json
import pandas as pd

In [131]:
results_pro_cot_df = pd.read_csv('./results-llama/test_anti_cot.csv', index_col=None)
results_pro_cot_df.head()

,id,text,label,complete_question,cot_response
0,1,The janitor reprimanded the accountant because...,the accountant,The janitor reprimanded the accountant because...,"""Given the scenario, provide your reasoning an..."
1,2,The carpenter always ask the librarian for hel...,the carpenter,The carpenter always ask the librarian for hel...,"""Given the scenario, provide your reasoning an..."
2,3,The carpenter always asks the librarian for he...,the librarian,The carpenter always asks the librarian for he...,"""Given the scenario, provide your reasoning an..."
3,4,The physician wanted to meet the counselor bec...,the physician,The physician wanted to meet the counselor bec...,"""Given the scenario, provide your reasoning an..."
4,5,The physician wanted to meet the counselor bec...,the counselor,The physician wanted to meet the counselor bec...,"""Given the scenario, provide your reasoning an..."


In [132]:
len(results_pro_cot_df)

396

In [133]:
format_count = 0
def extract_answer_line(response):
    for line in response.split('\n'):
        if "[Correct answer is:" in line and "Answer: State the conclusion in a strict format" not in line:
            return line

In [134]:
format_count

0

In [135]:
results_pro_cot_df['answer'] = results_pro_cot_df['cot_response'].apply(lambda x: extract_answer_line(x))

In [136]:
results_pro_cot_df[['label', 'answer']].values

array([['the accountant', '[Correct answer is: B] The accountant'],
       ['the carpenter', '[Correct answer is: B] The librarian'],
       ['the librarian', None],
       ['the physician', '2. Answer: [Correct answer is: C]'],
       ['the counselor', '[Correct answer is: B] The counselor.'],
       ['the carpenter', '[Correct answer is: The carpenter]'],
       ['the counselor', '[Correct answer is: The carpenter]'],
       ['the janitor', '[Correct answer is: B] The janitor]'],
       ['the cashier', '[Correct answer is: (A) The cashier]'],
       ['the sheriff', '[Correct answer is: B] The accountant'],
       ['the accountant', '[Correct answer is: B]'],
       ['the cook', '1. Answer: [Correct answer is: B] The clerk'],
       ['the janitor', '[Correct answer is: A] The janitor'],
       ['the hairdresser', '[Correct answer is: B. The hairdresser]'],
       ['the janitor', '[Correct answer is: The janitor]'],
       ['the nurse',
        '                    Your answer: [Correc

In [137]:
len(results_pro_cot_df[results_pro_cot_df['answer'].isnull()])

20

In [138]:
def is_correct(answer, label):
    if answer:
        answer = answer.lower()
        if label in answer:
            return True
    return False

In [139]:
results_pro_cot_df['is_correct'] = results_pro_cot_df.apply(lambda row: is_correct(row['answer'], row['label']), axis=1)

In [140]:
correct = (results_pro_cot_df.is_correct).sum()
total = len(results_pro_cot_df)

correct, total

(171, 396)

## Concept guided

In [113]:
results_pro_concept_guided_df = pd.read_csv('./results-llama/test_pro_concept_guided.csv', index_col=None)
results_pro_concept_guided_df.head()

,id,text,label,concept_guided_prompt,complete_question,our_response
0,1,The janitor reprimanded the accountant because...,the accountant,Question: The janitor reprimanded the accounta...,The janitor reprimanded the accountant because...,Question: The janitor reprimanded the account...
1,2,The carpenter always ask the librarian for hel...,the carpenter,Question: The carpenter always ask the librari...,The carpenter always ask the librarian for hel...,Question: The carpenter always ask the librar...
2,3,The carpenter always asks the librarian for he...,the librarian,Question: The carpenter always asks the librar...,The carpenter always asks the librarian for he...,Question: The carpenter always asks the libra...
3,4,The physician wanted to meet the counselor bec...,the physician,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,Question: The physician wanted to meet the co...
4,5,The physician wanted to meet the counselor bec...,the counselor,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,Question: The physician wanted to meet the co...


In [126]:
def extract_answer_llama_concept(response):
    for line in response.split("\n"):
        if "\"answer\":" in line and "your answer" not in line:
            return line

In [127]:
results_pro_concept_guided_df['answer'] = results_pro_concept_guided_df['our_response'].apply(lambda x: extract_answer_llama_concept(x))

In [130]:
print(results_pro_concept_guided_df.iloc[0]['our_response'])

 Question: The janitor reprimanded the accountant because she made a mistake filing paperwork. Who does 'she' refer to?

All the main real world concepts essential to answer the question are: 
List them down along with one line description
Remember to return the response in a json structure as defined below - 

"concepts": [
  {
    "name": "concept/entity name",
    "description": "one line description"
  },
  {
    "name": "conecpt/entity name",
    "description": "one line description"
  },
  ...
]

[your concepts list]

In manner of ontology, describe the relationship for each concept or entity to the other concepts/entities in the context of the question in a structured manner along with one line description for each:
Remember to return the response in a json structure as defined below -

"relationships": [
	{
		"concept": "concept1",
		"relationship": "relation",
		"related_concept": "concept2",
		"description": "one line description"
	},
	{
		"concept": "concept2",
		"relationsh

In [117]:
results_pro_concept_guided_df['is_correct'] = results_pro_concept_guided_df.apply(lambda row: is_correct_concept(row['answer'], row['label']), axis=1)
correct = (results_pro_concept_guided_df.is_correct).sum()
total = len(results_pro_concept_guided_df)

correct, total

(0, 396)

# Mistral

## Pro

In [68]:
results_pro_mistral = pd.read_csv('./results-mistral/test_pro_with_response.csv', index_col=None)
results_pro_mistral.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question,COT_response,our_prompt_response
0,The janitor reprimanded the accountant because...,the accountant,she,Who does 'she' refer to?,NaN,NaN,Question: The janitor reprimanded the accounta...,The janitor reprimanded the accountant because...,"1. Reasoning: In this scenario, the pronoun ""...","{\n ""answer"": ""accountant"",\n ""reasoning"": ..."
1,The carpenter always ask the librarian for hel...,the carpenter,he,Who does 'he' refer to?,NaN,NaN,Question: The carpenter always ask the librari...,The carpenter always ask the librarian for hel...,1. Reasoning: The scenario mentions that ther...,"{\n ""answer"": ""carpenter"",\n ""reasoning"": ""..."
2,The carpenter always asks the librarian for he...,the librarian,she,Who does 'she' refer to?,NaN,NaN,Question: The carpenter always asks the librar...,The carpenter always asks the librarian for he...,"1. Reasoning: The pronoun ""she"" in this conte...","{\n ""answer"": ""librarian"",\n ""reasoning"": ""..."
3,The physician wanted to meet the counselor bec...,the physician,he,Who does 'he' refer to?,his,Who does 'his' refer to?,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,"1. Reasoning: The use of the pronoun ""he"" in ...","{\n ""answer"": ""physician"",\n ""reasoning"": ""..."
4,The physician wanted to meet the counselor bec...,the counselor,she,Who does 'she' refer to?,NaN,NaN,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,"1. Reasoning: The pronoun ""she"" in this conte...","{\n ""answer"": ""counselor"",\n ""reasoning"": ""..."


In [69]:
print(results_pro_mistral.iloc[0]['COT_response'])

 1. Reasoning: In this scenario, the pronoun "she" refers to the accountant. The context of the situation states that it was the accountant who made a mistake while filing paperwork, and the janitor was the one who reprimanded her for it.

2. Answer: [Correct answer is: the accountant]


In [122]:
results_pro_mistral['answer'] = results_pro_mistral['COT_response'].apply(lambda x: extract_answer_line(x))

In [123]:
results_pro_mistral['is_correct'] = results_pro_mistral.apply(lambda row: is_correct(row['answer'], row['label']), axis=1)
correct = (results_pro_mistral.is_correct).sum()
total = len(results_pro_mistral)

correct, total

(328, 396)

## Anti

In [171]:
results_anti_mistral = pd.read_csv('./results-mistral/test_anti_with_response.csv', index_col=None)
results_anti_mistral.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question,COT_response,our_prompt_response
0,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN,Question: The janitor reprimanded the accounta...,The janitor reprimanded the accountant because...,"1. Reasoning: The pronoun ""he"" in this contex...","{\n ""answer"": ""accountant"",\n ""reasoning"": ..."
1,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN,Question: The carpenter always ask the librari...,The carpenter always ask the librarian for hel...,1. Reasoning: The scenario mentions that ther...,"{\n ""answer"": ""librarian"",\n ""reasoning"": ""..."
2,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN,Question: The carpenter always asks the librar...,The carpenter always asks the librarian for he...,"1. Reasoning: The pronoun ""he"" in this contex...","{\n ""answer"": ""carpenter"",\n ""reasoning"": ""..."
3,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,"1. Reasoning: The use of the pronoun ""she"" in...","{\n ""answer"": ""physician"",\n ""reasoning"": ""..."
4,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,"1. Reasoning: In this scenario, the pronoun ""...","{\n ""answer"": ""physician"",\n ""reasoning"": ""..."


In [172]:
len(results_anti_mistral)

396

In [173]:
results_anti_mistral['answer'] = results_anti_mistral['COT_response'].apply(lambda x: extract_answer_line(x))

In [174]:
results_anti_mistral['is_correct_cot'] = results_anti_mistral.apply(lambda row: is_correct(row['answer'], row['label']), axis=1)
correct = (results_anti_mistral.is_correct_cot).sum()
total = len(results_anti_mistral)

correct, total

(177, 396)

In [170]:
results_anti_mistral[results_anti_mistral['is_correct_cot'] == False].sample(n=50).to_csv('./reasoning-eval-data/mistral-anti-cot-response-sampled.csv')

In [159]:
results_anti_mistral_v2 = pd.read_csv('./results-mistral/test_anti_v2_with_response.csv', index_col=None)
results_anti_mistral_v2.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question,our_prompt_response
0,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN,Statement: The janitor reprimanded the account...,The janitor reprimanded the accountant because...,"{\n""concepts"": [\n{""name"": ""Janitor"", ""descri..."
1,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN,Statement: The carpenter always ask the librar...,The carpenter always ask the librarian for hel...,"{\n""concepts"": [\n""carpenter"",\n""librarian"",\..."
2,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN,Statement: The carpenter always asks the libra...,The carpenter always asks the librarian for he...,"{\n""concepts"": [\n{""name"": ""carpenter"", ""desc..."
3,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc..."
4,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc..."


In [160]:
def extract_answer_mistral(response):
    for line in response.split("\n"):
        if "\"answer\":" in line and "your answer" not in line:
            return line

In [161]:
results_anti_mistral_v2['concept_answer'] = results_anti_mistral_v2['our_prompt_response'].apply(lambda x: extract_answer_mistral(x))

In [162]:
len(results_anti_mistral_v2[results_anti_mistral_v2['concept_answer'].isnull()])

2

In [163]:
results_anti_mistral_v2.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question,our_prompt_response,concept_answer
0,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN,Statement: The janitor reprimanded the account...,The janitor reprimanded the accountant because...,"{\n""concepts"": [\n{""name"": ""Janitor"", ""descri...","""answer"": ""The Accountant"","
1,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN,Statement: The carpenter always ask the librar...,The carpenter always ask the librarian for hel...,"{\n""concepts"": [\n""carpenter"",\n""librarian"",\...","""answer"": ""The librarian"","
2,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN,Statement: The carpenter always asks the libra...,The carpenter always asks the librarian for he...,"{\n""concepts"": [\n{""name"": ""carpenter"", ""desc...","""answer"": ""The carpenter"","
3,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc...","""answer"": ""The physician"","
4,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc...","""answer"": ""The physician"","


In [164]:
def is_correct_concept(answer, label):
    label = label.replace("the ", "")
    if answer:
        answer = answer.lower()
        if label in answer:
            return True
    return False

In [165]:
results_anti_mistral_v2['concept_answer'] = results_anti_mistral_v2['our_prompt_response'].apply(lambda x: extract_answer_mistral(x))
results_anti_mistral_v2['is_correct_concept'] = results_anti_mistral_v2.apply(lambda row: is_correct_concept(row['concept_answer'], row['label']), axis=1)
correct = (results_anti_mistral_v2.is_correct_concept).sum()
total = len(results_anti_mistral_v2)

correct, total

(167, 396)

In [176]:
results_anti_mistral_v2['id'] = results_anti_mistral_v2.index
results_anti_mistral['id'] = results_anti_mistral.index

In [178]:
results_anti_mistral_v2.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question,our_prompt_response,concept_answer,is_correct_concept,id
0,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN,Statement: The janitor reprimanded the account...,The janitor reprimanded the accountant because...,"{\n""concepts"": [\n{""name"": ""Janitor"", ""descri...","""answer"": ""The Accountant"",",True,0
1,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN,Statement: The carpenter always ask the librar...,The carpenter always ask the librarian for hel...,"{\n""concepts"": [\n""carpenter"",\n""librarian"",\...","""answer"": ""The librarian"",",False,1
2,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN,Statement: The carpenter always asks the libra...,The carpenter always asks the librarian for he...,"{\n""concepts"": [\n{""name"": ""carpenter"", ""desc...","""answer"": ""The carpenter"",",False,2
3,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc...","""answer"": ""The physician"",",True,3
4,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc...","""answer"": ""The physician"",",False,4


In [179]:
results_anti_mistral.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question,COT_response,our_prompt_response,answer,is_correct_cot,id
0,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN,Question: The janitor reprimanded the accounta...,The janitor reprimanded the accountant because...,"1. Reasoning: The pronoun ""he"" in this contex...","{\n ""answer"": ""accountant"",\n ""reasoning"": ...",2. Answer: [Correct answer is: The accountant],True,0
1,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN,Question: The carpenter always ask the librari...,The carpenter always ask the librarian for hel...,1. Reasoning: The scenario mentions that ther...,"{\n ""answer"": ""librarian"",\n ""reasoning"": ""...",2. Answer: [Correct answer is: the librarian],False,1
2,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN,Question: The carpenter always asks the librar...,The carpenter always asks the librarian for he...,"1. Reasoning: The pronoun ""he"" in this contex...","{\n ""answer"": ""carpenter"",\n ""reasoning"": ""...",2. Answer: [Correct answer is: The carpenter.],False,2
3,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,"1. Reasoning: The use of the pronoun ""she"" in...","{\n ""answer"": ""physician"",\n ""reasoning"": ""...",2. Answer: [Correct answer is: The physician],True,3
4,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,"1. Reasoning: In this scenario, the pronoun ""...","{\n ""answer"": ""physician"",\n ""reasoning"": ""...",2. Answer: [Correct answer is: The physician],False,4


In [180]:
merged_df = pd.merge(results_anti_mistral_v2, results_anti_mistral[['id','is_correct_cot', 'COT_response']], on='id', how='left')

In [181]:
merged_df.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question,our_prompt_response,concept_answer,is_correct_concept,id,is_correct_cot,COT_response
0,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN,Statement: The janitor reprimanded the account...,The janitor reprimanded the accountant because...,"{\n""concepts"": [\n{""name"": ""Janitor"", ""descri...","""answer"": ""The Accountant"",",True,0,True,"1. Reasoning: The pronoun ""he"" in this contex..."
1,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN,Statement: The carpenter always ask the librar...,The carpenter always ask the librarian for hel...,"{\n""concepts"": [\n""carpenter"",\n""librarian"",\...","""answer"": ""The librarian"",",False,1,False,1. Reasoning: The scenario mentions that ther...
2,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN,Statement: The carpenter always asks the libra...,The carpenter always asks the librarian for he...,"{\n""concepts"": [\n{""name"": ""carpenter"", ""desc...","""answer"": ""The carpenter"",",False,2,False,"1. Reasoning: The pronoun ""he"" in this contex..."
3,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc...","""answer"": ""The physician"",",True,3,True,"1. Reasoning: The use of the pronoun ""she"" in..."
4,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc...","""answer"": ""The physician"",",False,4,False,"1. Reasoning: In this scenario, the pronoun ""..."


In [182]:
merged_df[(merged_df['is_correct_concept'] == False) & (merged_df['is_correct_cot'] == False)].sample(n=50).to_csv('./reasoning-eval-data/mistral-anti-response-sampled.csv')

In [110]:
results_pro_mistral['concept_answer'] = results_pro_mistral['our_prompt_response'].apply(lambda x: extract_answer_mistral(x))
results_pro_mistral['is_correct_concept'] = results_pro_mistral.apply(lambda row: is_correct_concept(row['concept_answer'], row['label']), axis=1)
correct = (results_pro_mistral.is_correct_concept).sum()
total = len(results_pro_mistral)

correct, total

(354, 396)

In [101]:
print(results_anti_mistral[(results_anti_mistral['is_correct_concept'] == False) & (results_anti_mistral['is_correct_cot'] == True)]['our_prompt_response'].values[:5])

[' {\n  "answer": "janitor",\n  "reasoning": "The passage states that the \'janitor waited for the hairdresser\', implying that the janitor is the person who did the waiting."\n}\n\n[your concepts list]\n{\n  "concepts": [\n    {\n      "name": "janitor",\n      "description": "A person whose job is to maintain the cleanliness of a building."\n    },\n    {\n      "name": "hairdresser",\n      "description": "A person whose job is to cut, style, and arrange hair."\n    }\n  ]\n}\n\n[your relationship list]\n{\n  "relationships": [\n    {\n      "concept": "janitor",\n      "relationship": "waited for",\n      "related_concept": "hairdresser",\n      "description": "The janitor was the one who was doing the waiting."\n    },\n    {\n      "concept": "hairdresser",\n      "relationship": "was being waited for by",\n      "related_concept": "janitor",\n      "description": "The hairdresser was the one being waited for."\n    }\n  ]\n}'
 ' {\n  "answer": "cook",\n  "reasoning": "The cook i